In [50]:
import pandas as pd
import pandas as pd
import torch
from datetime import datetime

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback

from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)






In [4]:
!pip install mlflow dagshub

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 31.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.7/233.7 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 41.4 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s et

In [9]:
import pandas as pd



2024/06/29 11:50:10 ERROR : Daemon timed out. Failed to terminate daemon pid 268: os: process already finished
2024/06/29 11:50:10 Fatal error: daemon exited with error code 1


In [43]:



df_train = pd.read_csv(r'/kaggle/input/sample-3000/train_df_2100_manual_final.csv')
df_val = pd.read_csv(r'/kaggle/input/sample-3000/validation_df_450_manual_final.csv')
df_test = pd.read_csv(r'/kaggle/input/sample-3000/test_df_450_manual_final.csv')



total_df = pd.concat([df_train, df_val, df_test], axis=0)
total_df.shape

df_train.rename(columns={'manual labeling': 'manual_labeling'}, inplace=True)
df_val.rename(columns={'manual labeling': 'manual_labeling'}, inplace=True)
df_test.rename(columns={'manual labeling': 'manual_labeling'}, inplace=True)
total_df.rename(columns={'manual labeling': 'manual_labeling'}, inplace=True)


df_train['manual_labeling_new'] = df_train['manual_labeling'].replace({-1: 0, 0: 1, 1: 2})
df_test['manual_labeling_new'] = df_test['manual_labeling'].replace({-1: 0, 0: 1, 1: 2})
df_val['manual_labeling_new'] = df_val['manual_labeling'].replace({-1: 0, 0: 1, 1: 2})

total_df['manual_labeling_new'] = total_df['manual_labeling'].replace({-1: 0, 0: 1, 1: 2})


In [44]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

In [45]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',  # Padding
            truncation=True,       # Truncation
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)  # Ensure labels are tensors of type long
        }


In [14]:
# Extract texts and labels
train_texts = df_train['comment_processed_text'].tolist()
train_labels = df_train['manual_labeling_new'].tolist()
val_texts = df_val['comment_processed_text'].tolist()
val_labels = df_val['manual_labeling_new'].tolist()
test_texts = df_test['comment_processed_text'].tolist()
test_labels = df_test['manual_labeling_new'].tolist()

In [51]:
MAX_LEN = 512

# Create train, validation, and test datasets
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer, MAX_LEN)
test_dataset = SentimentDataset(test_texts, test_labels, tokenizer, MAX_LEN)

In [52]:
train_dataset[0]


{'text': 'uff lancha nueva',
 'input_ids': tensor([    4, 16397, 30977, 27666,  2035,     5,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
          

In [17]:
import os
# Ensure the output directory exists
output_dir = './results_beto'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Ensure the logging directory exists
logging_dir = './logs'
if not os.path.exists(logging_dir):
    os.makedirs(logging_dir)

In [53]:

model = BertForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pip install accelerate -U

In [ ]:
pip show accelerate


In [ ]:
#pip install transformers[torch]

In [21]:
# Define custom metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [29]:


# Training Arguments
training_args = TrainingArguments(
    output_dir=output_dir,  # Use the output directory variable
    num_train_epochs=20,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logging_dir,
    report_to=[],
)

In [30]:
# Data Collator
data_collator = DataCollatorWithPadding(tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Add custom metrics
    data_collator=data_collator       # Add data collator
)

In [54]:
# Training Arguments
training_args = TrainingArguments(
    output_dir=output_dir,  # Use the output directory variable
    num_train_epochs=10,  # Increase number of epochs
    per_device_train_batch_size=16,  # Adjust batch size
    per_device_eval_batch_size=16,
    learning_rate=2e-5,  # Adjust learning rate
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logging_dir,
    eval_strategy="epoch",  # Evaluate at each epoch
    save_strategy="epoch",  # Save at each epoch
    save_total_limit=3,  # Only keep the last 3 checkpoints
    load_best_model_at_end=True,  # Load the best model found during training at the end
    metric_for_best_model="accuracy",  # Use accuracy to determine the best model
    greater_is_better=True,
    report_to=[],  # Disable all integrations
)

In [55]:
# Data Collator
data_collator = DataCollatorWithPadding(tokenizer)
# Trainer with Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,  # Add custom metrics
    data_collator=data_collator,       # Add data collator
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Add early stopping
)


In [56]:
# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.019317,0.526667,0.475056,0.587979,0.526667
2,No log,0.737039,0.680000,0.653878,0.711283,0.680000
3,No log,0.653039,0.722222,0.715969,0.718083,0.722222
4,No log,0.635270,0.726667,0.729006,0.739699,0.726667
5,No log,0.726107,0.713333,0.712583,0.718162,0.713333
6,No log,0.725617,0.724444,0.724921,0.726731,0.724444
7,No log,0.915165,0.708889,0.711573,0.716784,0.708889


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=462, training_loss=0.5870343493176745, metrics={'train_runtime': 803.7606, 'train_samples_per_second': 26.127, 'train_steps_per_second': 0.821, 'total_flos': 3867767240601600.0, 'train_loss': 0.5870343493176745, 'epoch': 7.0})

In [57]:
# Evaluate the model on the validation set
val_results = trainer.evaluate()
print("Validation Results:", val_results)

# Evaluate the model on the test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Test Results:", test_results)



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Validation Results: {'eval_loss': 0.6352695226669312, 'eval_accuracy': 0.7266666666666667, 'eval_f1': 0.7290055924122579, 'eval_precision': 0.7396993251501143, 'eval_recall': 0.7266666666666667, 'eval_runtime': 7.9827, 'eval_samples_per_second': 56.372, 'eval_steps_per_second': 1.879, 'epoch': 7.0}


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Test Results: {'eval_loss': 0.605351984500885, 'eval_accuracy': 0.7733333333333333, 'eval_f1': 0.7731599172459577, 'eval_precision': 0.7733569269335693, 'eval_recall': 0.7733333333333333, 'eval_runtime': 8.0612, 'eval_samples_per_second': 55.823, 'eval_steps_per_second': 1.861, 'epoch': 7.0}


In [60]:
val_results

{'eval_loss': 0.6352695226669312,
 'eval_accuracy': 0.7266666666666667,
 'eval_f1': 0.7290055924122579,
 'eval_precision': 0.7396993251501143,
 'eval_recall': 0.7266666666666667,
 'eval_runtime': 7.9827,
 'eval_samples_per_second': 56.372,
 'eval_steps_per_second': 1.879,
 'epoch': 7.0}

In [61]:
test_results

{'eval_loss': 0.605351984500885,
 'eval_accuracy': 0.7733333333333333,
 'eval_f1': 0.7731599172459577,
 'eval_precision': 0.7733569269335693,
 'eval_recall': 0.7733333333333333,
 'eval_runtime': 8.0612,
 'eval_samples_per_second': 55.823,
 'eval_steps_per_second': 1.861,
 'epoch': 7.0}

In [62]:
# Generate and print the classification report for validation set
preds_val = trainer.predict(val_dataset)
labels_val = preds_val.label_ids
predictions_val = preds_val.predictions.argmax(-1)
print("\nClassification Report (Validation):")
print(classification_report(labels_val, predictions_val, target_names=['negative', 'neutral', 'positive']))

# Generate and print the classification report for test set
preds_test = trainer.predict(test_dataset)
labels_test = preds_test.label_ids
predictions_test = preds_test.predictions.argmax(-1)
print("\nClassification Report (Test):")
print(classification_report(labels_test, predictions_test, target_names=['negative', 'neutral', 'positive']))

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



Classification Report (Validation):
              precision    recall  f1-score   support

    negative       0.81      0.67      0.74       181
     neutral       0.59      0.70      0.64       138
    positive       0.79      0.82      0.81       131

    accuracy                           0.73       450
   macro avg       0.73      0.73      0.73       450
weighted avg       0.74      0.73      0.73       450



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



Classification Report (Test):
              precision    recall  f1-score   support

    negative       0.82      0.80      0.81       181
     neutral       0.69      0.69      0.69       138
    positive       0.80      0.83      0.81       131

    accuracy                           0.77       450
   macro avg       0.77      0.77      0.77       450
weighted avg       0.77      0.77      0.77       450



In [64]:
# Define the paths where you want to save the model and tokenizer
import os

model_save_path = './saved_model'
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

tokenizer_save_path = './saved_tokenizer'
if not os.path.exists(tokenizer_save_path):
    os.makedirs(tokenizer_save_path)

    



In [41]:
# Save the model
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_save_path)

('./saved_tokenizer/tokenizer_config.json',
 './saved_tokenizer/special_tokens_map.json',
 './saved_tokenizer/vocab.txt',
 './saved_tokenizer/added_tokens.json')

In [36]:
pip install datetime

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 766.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 3.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [38]:
# Save the model and tokenizer with a unique name
import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_save_path = f'./saved_model_{timestamp}'
tokenizer_save_path = f'./saved_tokenizer_{timestamp}'

if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)
if not os.path.exists(tokenizer_save_path):
    os.makedirs(tokenizer_save_path)

# Save the model
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(tokenizer_save_path)

AttributeError: module 'datetime' has no attribute 'now'